<a href="https://colab.research.google.com/github/spencer18001/machine-learning-zoomcamp/blob/main/05/hw_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%capture
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/bank-full.csv

--2024-10-25 10:22:16--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/bank-full.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/bank-full.csv [following]
--2024-10-25 10:22:16--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/bank-full.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4610348 (4.4M) [text/plain]
Saving to: ‘bank-full.csv’

bank-full.csv       100%[===================>]   4.40M  --.-KB/s    in 0.09s   

2024-10-25 10:22:17 (47.4 MB/s) - ‘bank-full.csv’ saved [4610348/4610348]



In [2]:
!pipenv --version #Q1

pipenv, version 2024.2.0


In [5]:
%%capture
!pipenv install scikit-learn==1.5.2

To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing scikit-learn==1.5.2...
✔ Installation Succeeded
Building requirements...
Resolving dependencies...
✔ Success!
⠙ Locking packages...
Building requirements...
Resolving dependencies...
✔ Success!
⠙ Locking packages...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (0b43dd)...
All dependencies are now up-to-date!
Installing dependencies from Pipfile.lock (0b43dd)...


In [6]:
!grep -A 5 'scikit-learn' Pipfile.lock # Q2

        "scikit-learn": {
            "hashes": [
                "sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445",
                "sha256:178ddd0a5cb0044464fc1bfc4cca5b1833bfc7bb022d70b05db8530da4bb3dd3",
                "sha256:1ff45e26928d3b4eb767a8f14a9a6efbf1cbff7c05d1fb0f95f211a89fd4f5de",
                "sha256:299406827fb9a4f862626d0fe6c122f5f87f8910b86fe5daa4c32dcd742139b6",


In [11]:
%%capture
PREFIX="https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework"
!wget $PREFIX/model1.bin
!wget $PREFIX/dv.bin

In [18]:
import pickle

with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)

In [21]:
customer = {"job": "management", "duration": 400, "poutcome": "success"}

In [23]:
X = dv.transform([customer])
model.predict_proba(X)[0, 1] # Q3

0.7590966516879658

In [24]:
%%capture
!pip install Flask gunicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.6 MB/s eta 0:00:00


In [29]:
%%capture
!wget -O hw_05_predict.py https://raw.githubusercontent.com/spencer18001/machine-learning-zoomcamp/main/05/hw_05_predict.py

In [31]:
import subprocess

process = subprocess.Popen(['gunicorn', '--bind', '0.0.0.0:9696', 'hw_05_predict:app'])
process.pid

12329

In [32]:
import requests

url = "http://0.0.0.0:9696/predict"
client = {"job": "student", "duration": 280, "poutcome": "failure"}
requests.post(url, json=client).json() # Q4

{'subscription': False, 'subscription_probability': 0.33480703475511053}